Set Up the Credentials
-----------------------
Set the up the credentials to submit the runs to `r1z1` cluster

This step is not needed in Databricks Notebook

In [4]:
# The CREDENTIALS has been preset with `conda env config vars set CREDENTIALS="xxs"`

# To generate CREDENTIALS for your local environment, you can use the following command:
# data = {
#     "workspace_url": "https://dbc-04ac0685-8857.staging.cloud.databricks.com/",
#     "token": "dapi338xx", 
#     "mosaic_token": "Y4x7xx",
# }
# data = json.dumps(data)
# credentials = base64.b64encode(data.encode('utf-8')).decode('utf-8')

import os
from ygong.mosaic import submit, _set_up_environment

credentials = os.environ.get("CREDENTIALS")
_set_up_environment(credentials)

Local Testing 
------------------
Run the trainer.fit locally

In [5]:
import sys
import importlib
sys.path.append(os.path.abspath('../src'))

# always reload for local updates to take effect
import train
importlib.reload(train)


<module 'train' from '/Users/yu.gong/workspace/mosaic/custom-train-demo/src/train.py'>

In [7]:
from dataclasses import asdict
import json
name = "custom-train-demo"
config = train.MyConfig(global_train_batch_size=4, name=name)
json_str = json.dumps(asdict(config))
print(json_str)
# train.main(config)

{"name": "custom-train-demo", "seed": 42, "dist_timeout": 600.0, "global_train_batch_size": 4, "device_train_microbatch_size": 16}


In [18]:
train.main(config)
print("Done")

Done


Submit Run 
----------

run the local train.py



In [ ]:
# git clone -b prototype "https://github.com/ygong1/llm-foundry.git" llm-foundry
# cd llm-foundry && pip install -e .[gpu] && cd ..
# git clone https://github.com/ygong1/custom-train-demo.git
# ../llm-foundry/scripts/train/launcher.py ./src/train.py  '{"seed": 42, "dist_timeout": 600.0, "global_train_batch_size": 1024, "device_train_microbatch_size": 16}'

In [10]:
from mcli import RunConfig
from ygong.mosaic import ScalingConfig
from ygong.mosaic import submit

config = train.MyConfig(global_train_batch_size=4, name=name)
json_str = json.dumps(asdict(config))
commands = [
    "cd ~/custom-train-demo",
    f'''~/llm-foundry/scripts/train/launcher.py ./src/train.py  '{json_str}' '''
]

scalingConfig = ScalingConfig(gpusNum=8, gpuType="a100_80gb", poolName="r1z1")

config = RunConfig(
    name="custom-train-demo",
    image='mosaicml/llm-foundry:2.2.1_cu121_flash2-latest',
    command="\n".join(commands),
    compute=scalingConfig.toCompute,
    integrations=[
                {
                   'integration_type': 'git_repo',
                   'git_repo': 'ygong1/llm-foundry',
                   'path': '~/llm-foundry',
                   'git_branch': 'prototype',
                   'pip_install': '-e .[gpu]',
                   'ssh_clone': False
                },
                {
                   'integration_type': 'git_repo',
                   'git_repo': 'ygong1/custom-train-demo',
                   'path': '~/custom-train-demo',
                   'ssh_clone': False
                },
                {
                   'integration_type': 'pip_packages',
                   'packages': ['pynvml', 'mosaicml-streaming[databricks]'],
                },
    ],
    env_variables={},
)




In [11]:
submit(None, config, scalingConfig, debug=True)

Button(description='cancel the run', style=ButtonStyle())

,Run Name,Run ID,Status,Experiment Run
0,custom-train-demo-oL1IFf,31bc887b-af62-4081-8026-3ad670c742f4,RUNNING,


2024-03-24 16:24:12,290 - ygong.mosaic.submit - DEBUG - waiting for the MLFLow experiment run to be ready, run statusRUNNING
2024-03-24 16:24:12,290 - ygong.mosaic.submit - DEBUG - waiting for the MLFLow experiment run to be ready, run statusRUNNING
2024-03-24 16:24:33,057 - ygong.mosaic.submit - DEBUG - run custom-train-demo-oL1IFf is terminated. Status TERMINATING
2024-03-24 16:24:33,057 - ygong.mosaic.submit - DEBUG - run custom-train-demo-oL1IFf is terminated. Status TERMINATING


Run(run_uid='31bc887b-af62-4081-8026-3ad670c742f4', name='custom-train-demo-oL1IFf', status=<RunStatus.TERMINATING: 'TERMINATING'>, created_at=datetime.datetime(2024, 3, 24, 23, 23, 33, 609000, tzinfo=datetime.timezone.utc), updated_at=datetime.datetime(2024, 3, 24, 23, 23, 33, 609000, tzinfo=datetime.timezone.utc), created_by='yu.gong@databricks.com', priority='MEDIUM', preemptible=False, retry_on_system_failure=False, cluster='r1z1', gpus=8, gpu_type='a100_80gb', cpus=0, node_count=1, latest_resumption=Resumption(index=0, cluster='r1z1', gpus=8, cpus=0, gpu_type='a100_80gb', node_count=1, status=<RunStatus.TERMINATING: 'TERMINATING'>, estimated_end_time=None, started_at=datetime.datetime(2024, 3, 24, 23, 23, 42, tzinfo=datetime.timezone.utc), ended_at=datetime.datetime(2024, 3, 24, 23, 24, 19, tzinfo=datetime.timezone.utc), reason=''), is_deleted=False, run_type=<RunType.TRAINING: 'TRAINING'>, max_retries=0, reason='', nodes=[Node(rank=0, name='a100-80sxm-h11-01', status='FAILED', re